In [1]:
%load_ext lab_black

In [2]:
import math
import os
import random
import shutil
import json
import logging
import sys
from collections import defaultdict
from datetime import datetime
from io import TextIOWrapper
from pathlib import Path
from typing import Callable, List, Sequence, Tuple

import albumentations as A
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import KFold
from torch import nn, optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, Subset
from torch.utils.tensorboard import SummaryWriter
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from tqdm import tqdm
from PIL import Image

import utils
import networks
from error_list import error_list

In [4]:
train_imgs = sorted(list(Path("data/box_effdet/train_imgs/").glob("*.jpg")))
test_imgs = sorted(list(Path("data/box_effdet/test_imgs/").glob("*.jpg")))

In [5]:
imgs = train_imgs + test_imgs

In [7]:
mean = [0, 0, 0]
for f in tqdm(imgs, ncols=100, file=sys.stdout):
    img = imageio.imread(f).astype(np.float64) / 255.0
    mean[0] += img[..., 0].mean()
    mean[1] += img[..., 1].mean()
    mean[2] += img[..., 2].mean()

100%|██████████████████████████████████████████████████████████| 5795/5795 [00:18<00:00, 308.00it/s]


In [10]:
mean[0] /= len(imgs)
mean[1] /= len(imgs)
mean[2] /= len(imgs)

In [11]:
mean

[0.4112477475044492, 0.41951101162188087, 0.41573220158294094]